In [ ]:
%pip uninstall -y spacy thinc numpy cymem preshed srsly murmurhash catalogue

%pip install -q numpy==1.26.4 spacy==3.7.4 thinc==8.2.2
!python -m spacy download en_core_web_sm

import os
os.kill(os.getpid(), 9)

Found existing installation: spacy 3.8.7
Uninstalling spacy-3.8.7:
  Successfully uninstalled spacy-3.8.7
Found existing installation: thinc 8.3.6
Uninstalling thinc-8.3.6:
  Successfully uninstalled thinc-8.3.6
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: cymem 2.0.11
Uninstalling cymem-2.0.11:
  Successfully uninstalled cymem-2.0.11
Found existing installation: preshed 3.0.10
Uninstalling preshed-3.0.10:
  Successfully uninstalled preshed-3.0.10
Found existing installation: srsly 2.5.1
Uninstalling srsly-2.5.1:
  Successfully uninstalled srsly-2.5.1
Found existing installation: murmurhash 1.0.13
Uninstalling murmurhash-1.0.13:
  Successfully uninstalled murmurhash-1.0.13
Found existing installation: catalogue 2.0.10
Uninstalling catalogue-2.0.10:
  Successfully uninstalled catalogue-2.0.10
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy, spacy
print("NumPy:", numpy.__version__)           # 1.26.4
nlp = spacy.load("en_core_web_sm")
print("spaCy OK")

NumPy: 1.26.4
spaCy OK


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import json, time, uuid, re

Mounted at /content/drive


In [ ]:
VISUAL_POS = {"NOUN", "PROPN", "ADJ"}  # nouns, proper nouns, adjectives
STOP = set("""
background detail details thing things something anything nothing very really just much many
""".split())

# A small mood lexicon (expandable)
MOOD_VOCAB = {
    "moody","cinematic","dreamy","vintage","nostalgic","vibrant","muted","pastel","surreal",
    "dramatic","high-contrast","low-contrast","gritty","soft","warm","cool","noir","minimal",
    "documentary","film-like","polaroid","watercolor","oil-painting","sketch","futuristic"
}

# Words commonly useful in image prompts (expandable)
VISUAL_HINTS = {
    "portrait","landscape","street","night","sunset","dawn","rain","fog","forest","mountain",
    "ocean","city","neon","bokeh","macro","aerial","long-exposure","wide-angle","studio","candid"
}

In [ ]:
# ===== Extract function =====
def extract_prompt(text: str, top_k_keywords: int = 12):
    doc = nlp(text)

    # base candidates: ADJ/NOUN/PROPN (visual words)
    candidates = []
    for tok in doc:
        if tok.pos_ in VISUAL_POS and tok.is_alpha:
            w = tok.lemma_.lower()
            if len(w) >= 3 and w not in STOP:
                candidates.append(w)

    # boost words that are likely visual/mood hints
    scored = {}
    for w in candidates:
        score = 1.0
        if w in MOOD_VOCAB: score += 2.0
        if w in VISUAL_HINTS: score += 1.5

        # simple weight: adjectives get a tiny boost
        score += 0.2 if w in MOOD_VOCAB else 0.0
        scored[w] = scored.get(w, 0) + score

    mood = [w for w in scored.keys() if w in MOOD_VOCAB]
    mood = sorted(set(mood), key=lambda x: -scored[x])[:5]  # up to 5 moods

    # pick top visual keywords (excluding mood terms to avoid duplication)
    for m in mood:
        scored.pop(m, None)
    keywords = sorted(scored.keys(), key=lambda x: -scored[x])[:top_k_keywords]

    # compose final prompt text (expandable)
    mood_phrase = ", ".join(mood) if mood else "cinematic"
    kw_phrase = ", ".join(keywords) if keywords else "soft light, clean background"
    final_prompt = f"{mood_phrase}, {kw_phrase}"

    return {
        "keywords": keywords,
        "mood": mood,
        "final_prompt": final_prompt
    }

In [ ]:
# ===== sample diary text =====
diary_text = """
The mountain peaks blushed with golden light as the sun set.
The town below glowed warmly against the winter chill.
In that moment, Banff felt like a storybook brought to life.
"""

result = extract_prompt(diary_text, top_k_keywords=12)
result

{'keywords': ['mountain',
  'peak',
  'golden',
  'light',
  'sun',
  'set',
  'town',
  'winter',
  'chill',
  'moment',
  'banff',
  'storybook'],
 'mood': [],
 'final_prompt': 'cinematic, mountain, peak, golden, light, sun, set, town, winter, chill, moment, banff, storybook'}

In [ ]:
# ===== Save to Google Drive (shared folder) =====
SHARED_DIR = "/content/drive/MyDrive/colab_shared/style_prompts"
import os
os.makedirs(SHARED_DIR, exist_ok=True)

uid = uuid.uuid4().hex[:8]
timestamp = int(time.time())
base = f"prompt_{timestamp}_{uid}"

json_path = f"{SHARED_DIR}/{base}.json"
with open(json_path, "w") as f:
    json.dump(result, f, indent=2)

txt_path = f"{SHARED_DIR}/{base}.txt"
with open(txt_path, "w") as f:
    f.write(result["final_prompt"])

print("Saved JSON:", json_path)
print("Saved TEXT:", txt_path)
print("Prompt:", result["final_prompt"])

Saved JSON: /content/drive/MyDrive/colab_shared/style_prompts/prompt_1755856296_067a399d.json
Saved TEXT: /content/drive/MyDrive/colab_shared/style_prompts/prompt_1755856296_067a399d.txt
Prompt: cinematic, mountain, peak, golden, light, sun, set, town, winter, chill, moment, banff, storybook
